# Packages

In [ ]:
import pandas as pd
import numpy as np
import sqlalchemy 
from sqlalchemy import create_engine, text
from routingpy import ORS as ors
import requests

from db_secrets import SQL_107
from ors_secrets import get_ors_key

# Connection

In [ ]:
## text for gp query
gp_query_text = """
SELECT p.[Organisation_Code]
      ,p.[Organisation_Name]
	  ,p.[Postcode]
FROM [UK_Health_Dimensions].[ODS].[GP_Practices_And_Prescribing_CCs_SCD] AS p
WHERE 
	1=1
	AND	p.[Is_Latest] = 1
	AND	p.[High_Level_Health_Authority_Code]  = 'QHM'
	AND	p.[Prescribing_Setting] = 4
	AND	p.[Parent_Organisation_Code] != '01H'
"""

In [ ]:
## text for site query
ae_query_text = """
-- removes messages
SET NOCOUNT ON

-- Creates lookup for UCC sites
DROP TABLE IF EXISTS #RXP_UCC
CREATE TABLE #RXP_UCC (
		[ods_code] varchar(5)
	,	[site_name] varchar(250)
	);
INSERT INTO #RXP_UCC ([ods_code],[site_name])
VALUES
	 ('RXPRD','Seaham Urgent Care Centre'			   )
	,('RXP09','Peterlee Urgent Care Centre'			   )
	,('RXP09','Peterlee UTC'						   )
	,('RXPCP','OOH University Hospital of North Durham')
	,('RXPCP','Durham UTC'							   )
	,('RXPDA','Darlington UTC'						   )
	,('RXPDA','Darlington Out of Hours Service'		   )
	,('RXPBA','Bishop Auckland Urgent Care Centre'	   )
	,('RXPBA','Bishop Auckland UTC'					   )
	,('RXP11','Shotley Bridge UTC'					   )
	,('RXP11','OOH Shotley Bridge Urgent Care Centre'  );

-- Gets postcodes
SELECT DISTINCT 
		a.[ods_code]
	,	t.[Organisation_Name]
	,	t.[Postcode]
FROM 
	(	SELECT DISTINCT
			[attendance.location.site] AS [ods_code]
		FROM
			[LocalDataNECS].[ecds].[emergency_care]	AS a
		WHERE
			1=1
			AND a.[attendance.location.department_type] in ('01','02','03','04')
			AND a.[attendance.location.hes_provider_3] in ('RTD','RR7','RTF','RXP','RVW','RTR','R0B','RNN')
			AND a.[attendance.arrival.date] >= '2022-01-1'
		UNION ALL
		SELECT
			[ods_code]
		FROM 
			#RXP_UCC AS u) AS a
	LEFT JOIN	[UK_Health_Dimensions].[ODS].[NHS_Trusts_And_Trust_Sites_SCD] AS t
			ON	t.[Is_Latest] = 1
			AND a.[ods_code] = t.[Organisation_Code];

DROP TABLE IF EXISTS #RXP_UCC;
"""

In [ ]:
## Create an engine + connection
engine = create_engine(SQL_107())
conn = engine.connect()

## Return data
df_gp_raw = pd.read_sql(gp_query_text,conn)
df_ae_raw = pd.read_sql(ae_query_text,conn)


In [ ]:
df_gp = df_gp_raw.copy()
df_ae = df_ae_raw.copy()

# Lat / Long

## API to get lat/long

In [ ]:
def fetch_lat_lon(data, col):
    
    # Convert the postcodes from the DataFrame into a list
    postcodes = data[col].tolist()

    # to split data into 100s
    n = round(data[col].count()/100)

    # results list
    results = []

    for x in range(0,n+1):
        
        # 100 rows
        start = x*100
        end = (x*100)+100
        
        # Prepare the payload for the POST request
        payload = {"postcodes": postcodes[start:end]}
        
        # Make the POST request to the API
        response = requests.post("https://api.postcodes.io/postcodes", json=payload)
        
        # Check if the response is successful
        if response.status_code == 200:
            print(f'Status 200. Fetched rows: {start} to {end-1}')

            # Parse the JSON response
            response_data = response.json()
            
            # Extract latitude and longitude from each result

            for i in response_data['result']:
                if i['result']:  # Ensure there is a valid result
                    lat = i['result']['latitude']
                    lon = i['result']['longitude']
                    results.append({"postcode": i['query']
                                    , "latitude": lat, "longitude": lon})
                else:
                    results.append({"postcode": i['query']
                                    , "latitude": None, "longitude": None})
        else:
            break

    # checks if last response was sucessful
    if response.status_code == 200:           
        # Convert results into a DataFrame
        results_df = pd.DataFrame(results).drop_duplicates()

        # joins results back into data
        data = data.merge(results_df, how = 'left'
                            ,left_on=col, right_on='postcode')
        return(data)
    
    # print error message if api fails
    else:
        print("Failed to fetch data:", response.status_code)
        return None

## Fetch lat/long

In [ ]:
# Fetch lat/lon for each postcode in the DataFrame
df_gp_geo = fetch_lat_lon(df_gp,'Postcode')
df_ae_geo = fetch_lat_lon(df_ae,'Postcode')

In [ ]:
df_gp_geo.head()

# Routing

In [ ]:
list_gp_geo = df_gp_geo[['longitude','latitude']].values.tolist()
list_ae_geo = df_ae_geo[['longitude','latitude']].values.tolist()


len_gp = len(list_gp_geo)
len_ae = len(list_ae_geo)
sources_list=list(range(0,len_gp))
destinations_list=list(range(len_gp,len_gp+len_ae))

list_geo = list_gp_geo + list_ae_geo

In [ ]:
ors_api = ors(api_key=get_ors_key())

In [ ]:
distance_matrix = ors_api.matrix(
             locations=list_geo
            ,profile='driving-car'
            ,sources=sources_list
            ,destinations=destinations_list            
            ,dry_run = False
        )
